 # Analysis 20190419
 * Written by: Chien-Cheng Shih
 * Running under conda env "lipidraft"
 * Source Images: testdata

In [0]:
# import resource
import subprocess
import os, sys
from core.fileop import DirCheck, ListFiles, GetPendingList, GetGrpFLs

#Functions ---------------------------------------------------------------
def dict2arg_fiji(arg):
    arg_str = str()
    for key, val in arg.items():
        str_tmp = str(key + '="' + val + '", ')
        arg_str = arg_str + str_tmp
    return arg_str

def list2arg_python(arg):
    arg_str = str()
    for var in arg:
        arg_str = arg_str +  "'" + str(var) + "' " 
    return arg_str

def list2arg_matlab(arg):
    arg_str = str()
    for i in range(len(arg)-1):
        arg_str = arg_str +  "'" + str(arg[i]) + "', " 
    arg_str = arg_str +  "'" + str(arg[-1]) + "'" 
    return arg_str
# -------------------------------------------------------------------------


 ## Specify Parameters
 * `fiji`: path to the fiji application
 * `codepath`: the workspace for code/script
 * `datapath`: the workspace for STORM images and data
 * `analysis_dir`: the folder hosting data and images generated from this pipeline

In [0]:
fiji = '/Applications/Fiji.app/Contents/MacOS/ImageJ-macosx'
matlab = '/Applications/MATLAB_R2017b.app/bin/matlab'
codepath = '/Users/michaelshih/Documents/code/wucci/storm_image_processing'
datapath = '/Volumes/LaCie_DataStorage/xiaochao_wei_STORM imaging/STORM_imaging'
analysis_dir = 'analysis_20190419'


 ## Script name: `imgproc.py`
 * env: ImageJ/Fiji
 * updated: 04/26/2019
 * verified
 * output folder: `preprocessing`
 ### Output
   1. `preproimg` (`.tif`): seperate the channels into individual folders
   2. `imgintensity` (`.csv`): return mean intensity for each time frame from the center of the image (128 pixel * 128 pixel)
   3. `imginfo`:
       1. `imgmetadata` (`.txt`): return the metadata
       2. `imgstat.csv`: statistics from images
           * image_name: the name of the image without extension
           * ip_file_name: the name of the image with extension
           * xSize: frame size on the x-axis
           * ySize: frame size on the y-axis
           * nSlices: frame size on the z-axis
           * nFrames: number of time frames
           * nChannels: number of channles
           * size_MB: size of the file

In [0]:
script_name = 'imgproc.py'
resource_img_dir = os.path.join('resource', 'testdata')
ippath = os.path.join(datapath, resource_img_dir)
opdir = 'preprocessing'
oppath = os.path.join(datapath, analysis_dir, opdir)

# create dict to store parameters
arg_dict = {
    'path': datapath,
    'dir_output': analysis_dir,
    'ippath': ippath,
    'outputdir': oppath,
    'batchmodeop': 'false',
}

arg = dict2arg_fiji(arg_dict)
print(arg)

In [0]:
# Run the script

In [0]:
subdir = 'Fiji'
scriptpath = os.path.join(codepath, subdir, script_name)
print("Start running: {}".format(script_name))
subprocess.check_output([fiji, '--ij2', '--run', scriptpath, arg])
print("End: {}".format(script_name))


 ## Script name: `tstormanalysis.py`
 * env: ImageJ/Fiji
 * plugin: [ThunderSTORM](https://github.com/zitmen/thunderstorm)
 * updated: 04/26/2019
 * verified
 * output folder: `tstorm`
 ### Output generated by ThunderSTORM
   1. `csvdata`:
       * (`.csv`): STORM dataset
       * (`.txt`): protocol
   2. `driftcorr` (`.json`): drift correction profile
   3. `histogram_plot` (`.tif`): histogram image with manification = 5.0

In [0]:
script_name = 'tstormanalysis.py'
ipdir = 'preprocessing'
ipsubdir = 'preproimg'
ippath = os.path.join(datapath, analysis_dir, ipdir, ipsubdir)

arg_dict = {
    'path': datapath,
    'dir_output': analysis_dir,
    'ippath': ippath,
    'batchmodeop': 'false', 
}
arg = dict2arg_fiji(arg_dict)
print(arg)


In [0]:
subdir = 'Fiji'
scriptpath = os.path.join(codepath, subdir, script_name)
print("Start running: {}".format(script_name))
subprocess.check_output([fiji, '--ij2', '--run', scriptpath, arg])
print("End: {}".format(script_name))


 ## Script name: `csv_slicer_T.py`
 * env: Python
 * updated: 04/26/2019
 * verified
 * output folder: `csvdata_sliced`
 ### Output generated by ThunderSTORM
   * `csvdata_sliced_T` (`.csv`): STORM data from T0 to T1

In [0]:
script_name = 'csv_slicer_T.py'
nchannel = 2
T0 = 5000
T1 = 10001
arg_list = [datapath, analysis_dir,'tstorm','csvdata', nchannel, T0, T1, 'csvdata_sliced_T']
arg = list2arg_python(arg_list)
print(arg)


In [0]:
print("Start running: {}".format(script_name))
shellcmd = str('python '+ os.path.join(codepath, script_name)+ ' '+ arg)
print(shellcmd)
process = subprocess.run(shellcmd, shell = True, check=True)
print("End: {}".format(script_name))


 ## Script name: `csv_slicer_ROI.py`
 * env: Python
 * updated: 04/29/2019
 * verified
 * output folder: `csvdata_sliced`
 ### Output generated by ThunderSTORM
   * `csvdata_sliced_T_ROI` (`.csv`): STORM data cropped by given region of interest.

In [0]:
# csv_slicer_crop.py
script_name = 'csv_slicer_ROI.py'
arg_list = [datapath, analysis_dir,'tstorm','csvdata_sliced_T', 2, 3, 'csvdata_sliced_T_ROI']
arg = list2arg_python(arg_list)
print(arg)


In [0]:
print("Start running: {}".format(script_name))
shellcmd = str('python '+ os.path.join(codepath, script_name)+ ' '+ arg)
print(shellcmd)
process = subprocess.run(shellcmd, shell = True, check=True)
print("End: {}".format(script_name))


In [0]:
# Spatial Analysis ============================================================================================================











 ## Script name: `spatialanaylsis.R`
 * env: R
 * updated: 04/29/2019
 * verified
 * output folder: `spacial_test`
## R, Spatstat
   * `spacialdata` (`.csv`): Results of Ripley's K
       * K_r: Ripley's K function $$\hat{K}(r)= \hat{\lambda}^{-1}\sum_{i}\sum_{j\neq1}\frac{\omega(l_i, l_j)I(d_{ij<r})}{N}$$
       * L_r: L-function $$\hat{L}(r)=\sqrt{\frac{\hat{K}(r)}{\pi}}$$
       * H_r: H-function $$\hat{H}(r)=\hat{L}(r)-r$$
   * `spacialdata_bi` (`.rda`): binary point pattern data

In [0]:
# spatialanaylsis.R
script_name = 'spatialanaylsis.R'
arg_list = [datapath, analysis_dir,'tstorm','csvdata_sliced_T_ROI', 2, 3, 'spacial_test', 'spacialdata', 'spacialdata_bi']
arg = list2arg_python(arg_list)
print(arg)


In [0]:
print("Start running: {}".format(script_name))
shellcmd = str('Rscript --vanilla '+ os.path.join(codepath, script_name)+ ' '+ arg)
print(shellcmd)
process = subprocess.run(shellcmd, shell = True, check=True)
print("End: {}".format(script_name))


 ## Script name: `plot_spatialdata.py`
 * env: Python
 * updated: 04/29/2019
 * verified
 * output folder: `spacial_test`
 ### Output generated by ThunderSTORM
   * `plot_K` (`.png`): Plot for r and K_r
   * `plot_L` (`.png`): Plot for r and L_r
   * `plot_H` (`.png`): Plot for r and H_r
   * `plot_total` (`.png`):
       * {function}_all_c{channel}.png: plots including all data in the given function (by files)
       * {function}_merge_c{channel}.png: plots including average data in the given function (by files)

In [0]:
script_name = 'plot_spatialdata.py'
arg_list = [datapath, analysis_dir,'spacial_test','spacialdata', 2, 'plot']
arg = list2arg_python(arg_list)
print(arg)


In [0]:
print("Start running: {}".format(script_name))
shellcmd = str('python '+ os.path.join(codepath, script_name)+ ' '+ arg)
print(shellcmd)
process = subprocess.run(shellcmd, shell = True, check=True)
print("End: {}".format(script_name))


 ## Script name: 'csv_slicer_ROI_pad.py'
 * env: Python
 * updated: 04/29/2019
 * verified
 * output folder: `csvdata_sliced_pad`
 ### Output generated by ThunderSTORM
   * `csvdata_sliced_T_ROI_pad` (`.csv`): STORM data cropped by given region of interest with defined pad region

In [0]:
# csv_slicer_ROI_pad.py
script_name = 'csv_slicer_ROI_pad.py'
arg_list = [datapath, analysis_dir,'tstorm','csvdata_sliced_T', 2, 3, 3, 'csvdata_sliced_T_ROI_pad']
arg = list2arg_python(arg_list)
print(arg)


In [0]:
print("Start running: {}".format(script_name))
shellcmd = str('python '+ os.path.join(codepath, script_name)+ ' '+ arg)
print(shellcmd)
process = subprocess.run(shellcmd, shell = True, check=True)
print("End: {}".format(script_name))



 ## Script name: `spatialanaylsis_localL_pad.R`
 * env: R
 * updated: 04/29/2019
 * verified
 * output folder: `spacial_test`
## R, Spatstat
   * `spacialdata` (`.csv`): Results of Ripley's K
       * K_r: Ripley's K function $$\hat{K}(r)= \hat{\lambda}^{-1}\sum_{i}\sum_{j\neq1}\frac{\omega(l_i, l_j)I(d_{ij<r})}{N}$$
       * L_r: L-function $$\hat{L}(r)=\sqrt{\frac{\hat{K}(r)}{\pi}}$$
       * H_r: H-function $$\hat{H}(r)=\hat{L}(r)-r$$
   * `spacialdata_bi` (`.rda`): binary point pattern data

In [0]:
# spatialanaylsis.R
script_name = 'spatialanaylsis_localL.R'
arg_list = [datapath, analysis_dir,'tstorm','csvdata_sliced_T_ROI_pad', 2, 3, 'spacial_test', 'spacialdata_pad_local']
arg = list2arg_python(arg_list)
print(arg)


In [0]:
print("Start running: {}".format(script_name))
shellcmd = str('Rscript --vanilla '+ os.path.join(codepath, script_name)+ ' '+ arg)
print(shellcmd)
process = subprocess.run(shellcmd, shell = True, check=True)
print("End: {}".format(script_name))


 ## Script name: `localL_interpolation_pad.m`
 * env: MATLAB
 * updated: 04/29/2019
 * verified
 <br />
 * Method: MATLAB, griddata, 'v4': Biharmonic spline interpolation (MATLAB® 4 griddata method)
 supporting 2-D interpolation only. Unlike the other methods, this interpolation is not based on a triangulation.
 * output folder: `spacialdata_pad_local_int`

In [0]:
script_name = 'localL_interpolation_pad.m'
arg_list = [codepath, datapath, analysis_dir, 'spacial_test', 'spacialdata_pad_local', 'spacialdata_pad_local_int', 'par', 'cropsize.csv', 160, 30, 3]
arg = list2arg_matlab(arg_list)
print(arg)


In [0]:
print("Start running: {}".format(script_name))
matlabfunc = script_name.replace('.m', '')
shellcmd = str(matlab + ' -r '+ '"' + matlabfunc + "(" + arg + ')"')
print(shellcmd)
process = subprocess.run(shellcmd, shell = True, check=True)
print("End: {}".format(script_name))


 ## Script name: `plot_localL_pad.py`
 * env: Python
 * updated: 04/29/2019
 * verified
 * output folder: `spacial_test`
 ### Output generated by ThunderSTORM
   1. 'plot_local_pad_scatter' (`.png`): scatter plot (with denisty in LUT)
   2. 'plot_local_pad_grid' (`.tif`): interpolated image (frame size: 480 * 480)

In [0]:
script_name = 'plot_localL_pad.py'
arg_list = [datapath, analysis_dir,'tstorm','spacial_test','spacialdata_pad_local', 'spacialdata_pad_local_int',  2, 3, 3, 'plot_local_pad_scatter', 'plot_local_pad_grid']
arg = list2arg_python(arg_list)
print(arg)


In [0]:
print("Start running: {}".format(script_name))
scriptpath = os.path.join(codepath, script_name)
shellcmd = str('python '+ scriptpath + ' '+ arg)
print(shellcmd)
process = subprocess.run(shellcmd, shell = True, check=True)
print("End: {}".format(script_name))


 ## Script name: `grid_cluster_analysis.py`
 * env: ImageJ/Fiji
 * updated: 04/26/2019
 * verified
 * output folder: `tstorm`
 ### Output generated by ThunderSTORM
   1. `int_grid_bi` (`.tif`): segemented image with clusters
   2. `int_grid_data` (`.csv`): label measurment from clusters

In [0]:
script_name = 'grid_cluster_analysis.py'
ippath = os.path.join(datapath, analysis_dir, 'spacial_test', 'plot_local_pad_grid')
opsub_dir = os.path.join('spacial_test', 'nnd')
op_bi_path = os.path.join(datapath, analysis_dir, opsub_dir, 'int_grid_bi')
op_data_path = os.path.join(datapath, analysis_dir, opsub_dir, 'int_grid_data')

arg_dict = {
    'path_j': datapath,
    'dir_output': analysis_dir,
    'ippath_j': ippath,
    'op_bi_path_j': op_bi_path,
    'op_data_path_j': op_data_path,
    'batchmodeop': 'false', 
}
arg = dict2arg_fiji(arg_dict)
print(arg)


In [0]:
subdir = 'Fiji'
scriptpath = os.path.join(codepath, subdir, script_name)
print("Start running: {}".format(script_name))
subprocess.check_output([fiji, '--ij2', '--run', scriptpath, arg])
print("End: {}".format(script_name))



 ## Script name: `nnd_cluster_prefilter.py`
 * env: Python
 * updated: 04/29/2019
 * verified
 * output folder: `spacial_test/nnd`
 ### Output generated by ThunderSTORM
   1. 'int_grid_data_filtered' (`.csv`): filter data (by area and circularity)
   2. 'int_grid_data_summary' (`.png`): histogram by area

In [0]:
script_name = 'nnd_cluster_prefilter.py'
arg_list = [datapath, analysis_dir,'tstorm','spacial_test','nnd', 'int_grid_data',  2, 3, 3, 'int_grid_data_summary', 'int_grid_data_filtered']
arg = list2arg_python(arg_list)
print(arg)


In [0]:
print("Start running: {}".format(script_name))
scriptpath = os.path.join(codepath, script_name)
shellcmd = str('python '+ scriptpath + ' '+ arg)
print(shellcmd)
process = subprocess.run(shellcmd, shell = True, check=True)
print("End: {}".format(script_name))


 ## Script name: `nnd_cluster_analysis.py`
 * env: Python
 * updated: 04/29/2019
 * verified
 * output folder: `spacial_test/nnd`
 ### Output generated by ThunderSTORM
   * 'int_grid_data_dist' (`.csv`): data of nnd processing

In [0]:
script_name = 'nnd_cluster_analysis.py'
arg_list = [datapath, analysis_dir,'tstorm','spacial_test','nnd', 'int_grid_data_filtered',  2, 3, 3, 'int_grid_data_dist']
arg = list2arg_python(arg_list)
print(arg)


In [0]:
print("Start running: {}".format(script_name))
scriptpath = os.path.join(codepath, script_name)
shellcmd = str('python '+ scriptpath + ' '+ arg)
print(shellcmd)
process = subprocess.run(shellcmd, shell = True, check=True)
print("End: {}".format(script_name))


 ## Script name: `nnd_plot.py`
 * env: Python
 * updated: 04/29/2019
 * verified
 * output folder: `spacial_test/nnd`
 ### Output generated by ThunderSTORM
   * 'int_grid_data_nndplot' (`.png`): plot nnd data
       1. summary_c{}_nnd.png: total nnd distribution
       2. summary_c{}_nnd_mean.png: nnd mean
       3. summary_c{}_nnd_nanocluster.png: nanocluster area and density

In [0]:
script_name = 'nnd_plot.py'
arg_list = [datapath, analysis_dir,'tstorm','spacial_test','nnd', 'int_grid_data_dist',  2, 3, 3, 0.01, 'int_grid_data_nndplot']
arg = list2arg_python(arg_list)
print(arg)


In [0]:
print("Start running: {}".format(script_name))
scriptpath = os.path.join(codepath, script_name)
shellcmd = str('python '+ scriptpath + ' '+ arg)
print(shellcmd)
process = subprocess.run(shellcmd, shell = True, check=True)
print("End: {}".format(script_name))


In [0]:
# CBC ============================================================================================================













 ## Script name: `CBC.py`
 * env: ImageJ/Fiji
 * plugin: [ThunderSTORM](https://github.com/zitmen/thunderstorm)
 * updated: 04/29/2019
 * verified
 * output folder: `CBC`
 ### Output generated by ThunderSTORM
   * CBC_results (`.csv`): CBC data

In [0]:
script_name = 'CBC.py'
ippath = os.path.join(datapath, analysis_dir, 'tstorm', 'csvdata_sliced_T_ROI')
opdir = 'CBC'
opsubdir = 'CBC_results'

arg_dict = {
    'path': datapath,
    'dir_output': analysis_dir,
    'ippath_j': ippath,
    'op_dir': opdir,
    'op_subdir': opsubdir,
    'batchmodeop': 'false', 
}
arg = dict2arg_fiji(arg_dict)
print(arg)


In [0]:
subdir = 'Fiji'
scriptpath = os.path.join(codepath, subdir, script_name)
print("Start running: {}".format(script_name))
subprocess.check_output([fiji, '--ij2', '--run', scriptpath, arg])
print("End: {}".format(script_name))


 ## Script name: `data_inference_cbc.py`
 * env: Python
 * updated: 04/29/2019
 * verified
 * output folder: `CBC`
 ### Output generated by ThunderSTORM
   1. 'CBC_histogram' (`.png`): individual histogram for CBC data
   2. 'CBC_histogram_summary' (`.png`): compiled histogram

In [0]:
script_name = 'data_inference_cbc.py'
arg_list = [datapath, analysis_dir,'CBC','CBC_results',  2, 'CBC_histogram', 'CBC_histogram_summary']
arg = list2arg_python(arg_list)
print(arg)


In [0]:
print("Start running: {}".format(script_name))
scriptpath = os.path.join(codepath, script_name)
shellcmd = str('python '+ scriptpath + ' '+ arg)
print(shellcmd)
process = subprocess.run(shellcmd, shell = True, check=True)
print("End: {}".format(script_name))


In [0]:
# data inference ============================================================================================================










 ## Script name: `data_inference_photon.py`
 * env: Python
 * updated: 04/30/2019
 * verified
 * output folder: `tstorm/stormdatainf_dir`
 ### Output generated by ThunderSTORM
   * photons_hist (`.csv`): histogram of photon for individual files
   * photons_hist_summary (`.png`): histogram of photon (total)

In [0]:
script_name = 'data_inference_photon.py'
arg_list = [datapath, analysis_dir,'tstorm', 'spacial_test', 'csvdata_sliced_T_ROI', 2, 'stormdatainf_dir', 'photons_hist_summary', 'photons_hist']
arg = list2arg_python(arg_list)
print(arg)


In [0]:
print("Start running: {}".format(script_name))
scriptpath = os.path.join(codepath, script_name)
shellcmd = str('python '+ scriptpath + ' '+ arg)
print(shellcmd)
process = subprocess.run(shellcmd, shell = True, check=True)
print("End: {}".format(script_name))


 ## Script name: `csv_slicer_th.py`
 * env: Python
 * updated: 04/30/2019
 * verified
 * output folder: `tstorm`
 ### Output generated by ThunderSTORM
   * csvdata_sliced_T_ROI_th (`.csv`): STORM data sliced by given threshold
   * csvdata_sliced_T_ROI_th_match (`.csv`): STORM data sliced by given threshold, selected with minimum counts

In [0]:
script_name = 'csv_slicer_th.py'
arg_list = [datapath, analysis_dir,'tstorm', 'spacial_test', 'csvdata_sliced_T_ROI', 2, 7000, 15000, 'csvdata_sliced_T_ROI_th', 50, 50, 'csvdata_sliced_T_ROI_th_match']
arg = list2arg_python(arg_list)
print(arg)


In [0]:
print("Start running: {}".format(script_name))
scriptpath = os.path.join(codepath, script_name)
shellcmd = str('python '+ scriptpath + ' '+ arg)
print(shellcmd)
process = subprocess.run(shellcmd, shell = True, check=True)
print("End: {}".format(script_name))

